In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import gif

In [2]:
class Individual:
    """ 
    A class to represent each indvidual in the SIR simulation 
    
    ...
    
    Attributes
    ----------
    identity: int
        This will be a unique integer to identify each individual. This is generally just what number the individual is out
        of the 'n' individuals in the simulation. 
    susceptible: boolean 
        Object is currently suceptible to infection
    infected: boolean
        object is currently infected and can infect others
    recovered: boolean
        object has recoverd and is no longer infectious - nor can it be infected again. 
    recovery_time: int
        how long it takes for the indivdual to recover (measured in frames)
    coord_x, coord_y: int
        The objects coordinates points on the grid
    x_low, x_high, y_low, y_high: int
        The grid boundaries 
        
    
    
    Methods
    -------
    setInfected(frame_num)
        Changes the object's status to infected. Also records the time point (frame) of infection. 
        
    setRecovered()
        Changes the object's status to recovered.
    
    setNewPosition()
        Determines the objects new position taking into account the object's speed. If the new position is 
        outside of the grid boundaries then the new position is set to the grid boundary. 
    
    getID()
        Returns the object's ID number
    
    getSIRstatus()
        Returns object's current SIR status. 
    
    getRecovered(frame_num)
        Taking into account the current frame number, this will determine if the object's should now change to 'recovered'.
        
    getPos()
        Returns object's coordinates on the grid 
        
    getDist(other_x, other_y)
        Returns the euclidean distance between the current object and another object. 
        
    getColor()
        Returns the colour associated with the Object's current SIR status
    
    """
    
    
    
    
    
    
    
    # Initialise Values for Individuals
    def __init__(self, identity, recovery_time, coord_x, coord_y, speed, x_low, x_high, y_low, y_high):
        
        """
        Parameters
        ----------
        identity: int
            This will be a unique integer to identify each individual. This is generally just what number the individual is out
            of the 'n' individuals in the simulation. 
        recovery_time: int
            how long it takes for the indivdual to recover (measured in frames)
        coord_x, coord_y: int
            The objects coordinates points on the grid
        x_low, x_high, y_low, y_high: int
            The grid boundaries 
        """
        
        # Identity 
        self.identity=identity
        
        
        # SIR status: Susceptible, Infection, or Recovered
        self.susceptible = True
        self.infected = False
        self.recovered = False
        # Recovery Time
        self.recovery_time = recovery_time
        # Time point of Infection
            # - We will later change this to zero once 'Individual' has become infected and the counter will begin.
        self.infect_frame = -1  
        
        
        # Current Position within the particle box
        self.coord_x = coord_x
        self.coord_y = coord_y
        # Speed
        self.speed = speed
        # Boundary Limits
        self.x_low = x_low
        self.x_high = x_high
        self.y_low = y_low
        self.y_high = y_high
        
        
    ### Setter Methods 
    
    # Set Status to Infected 
    def setInfected(self, frame_num):
        """  
        Changes the object's status to infected. Also records the time point (frame) of infection.
        """
        self.susceptible = False
        self.infected = True
        self.recovered = False
        self.infect_frame = frame_num
        
    # Set Status to Recovered
    def setRecovered(self):
        """
        Changes the object's status to recovered.
        """
        self.susceptible = False
        self.infected = False
        self.recovered = True
        
    # Set New position
    def setNewPosition(self):
        """
        Determines the objects new position taking into account the object's speed. If the new position is 
        outside of the grid boundaries then the new position is set to the grid boundary. 
        """
        tmp1 = np.random.randint(0,3)
        tmp2 = np.random.randint(0,3)
        
        # Move along the x-plane
        if tmp1 == 1:
            self.coord_x += self.speed
            if self.coord_x > self.x_high: self.coord_x=self.x_high
        if tmp1 == 2:
            self.coord_x -= self.speed
            if self.coord_x < self.x_low: self.coord_x=self.x_low
                    
        # Move along the y-plane
        if tmp2 == 1:
            self.coord_y += self.speed
            if self.coord_y > self.y_high: self.coord_y=self.y_high
        if tmp2 == 2:
            self.coord_y -= self.speed
            if self.coord_y < self.y_low: self.coord_y=self.y_low

    
    ### Getter Methods    
    
    # Returns 'Individuals' identity
    def getID(self):
        """
        Returns the object's ID number
        """
        return print(self.identity)
    
    # Returns Current SIR Status
    def getSIRstatus(self):
        """
        Returns object's current SIR status.
        """
        if self.susceptible:
            return "Susceptible"
        if self.infected:
            return "Infected"
        if self.recovered:
            return "Recovered"
    
    
    # Check whether the recovery period has finished
    def getRecovered(self, frame_num):
        """
        Taking into account the current frame number, this will determine if the object's should now change to 'recovered'.
        """
        if self.infect_frame > -1:
            if frame_num - self.infect_frame > self.recovery_time:
                self.setRecovered()
    
    # Return Current position
    def getPos(self):
        """
        Returns object's coordinates on the grid 
        """
        return (self.coord_x, self.coord_y)
    
    # Calculate distance between this 'Individual' and another
    def getDist(self, other_x, other_y):
        """
        Returns the euclidean distance between the current object and another object.
        """
        return np.sqrt(np.square(self.coord_x-other_x)+np.square(self.coord_y-other_y)) 
    
    # Returns SIR status colour 
    def getColor(self):
        """
        Returns the colour associated with the Object's current SIR status
        """
        if self.susceptible:
            return 'blue'
        if self.infected:
            return 'red'
        if self.recovered:
            return 'Green'    
        
        
        
        
        

In [10]:
### Implementation 

# Parameters
n=200  # num of individuals




inf_perc = 0.01  # initial % of infected
inf_array = np.linspace(0, 1, n) # an individuals infection will be set based on their index being above or below inf_perc
np.random.shuffle(inf_array) # ensuring infection is properly randomised 

inf_radius=2  # infection radius 
inf_prob=0.15  # infection probability when within inf_radius
recovery_time=100   

# grid limits
x_low= 0
x_high= 100
y_low=0
y_high=100



individuals=[] # list of instatiated Individuals

# Lists to store SIR numbers for each moment
global_S = []
global_I = []
global_R = []
global_Frame = []

# Number of SIR numbers for the initial frame. n_S will be defined after instantiation as n_S = n - n_I
n_I=0
n_R=0


# Instantiate Objects
for i in range(n):
    ind = Individual(identity=i, recovery_time=recovery_time, 
                     coord_x=np.random.random()*100, 
                     coord_y=np.random.random()*100, 
                     speed=np.random.random()*5, 
                     x_low=0, x_high=100, 
                     y_low=0, y_high=100)
    
    if inf_array[i] < inf_perc:
        ind.setInfected(frame_num=0)
        n_I+=1
    
    individuals.append(ind)
    
n_S=n-n_I # number of susceptible individuals. this is defined post instantiation as it has to account for those who were 
          # infected during instantiation. 

# the SIR numbers are appended to their respective lists to record the intial values. 
global_S.append(n_S)
global_I.append(n_I)
global_R.append(n_R)
global_Frame.append(0)

# Number of frames - ie time 
frames = [] # list to store the state of all objects plotted onto the grid 
num_frames=500


@gif.frame
def frame_plot(objects, x_low, x_high, y_low, y_high, time, sus, inf, rec):
    """
    Parameters
    ----------
    objects: list 
        list of instantiated 'Individual' objects
        
    x_low, x_high, y_low, y_high: int
        The grid boundaries 
        
    time: list 
        accepts the global_Frame list which it then plots as the x axis
        
    sus, inf, rec: list
        Each accepts their respective global list which is plotted on the y axis
    
    
    returns: 
        A graph with 2 subplots. The left plot is the particle in a box model. The plot on the right plots the changing 
        SIR numbers that occur within the particle in a box simulation. 
    """
    
    
    x=[]
    y=[]
    colour=[]
    
    for o in objects:
        x.append(o.getPos()[0])
        y.append(o.getPos()[1])
        colour.append(o.getColor())
        
    fig = plt.figure(figsize=(18,9))
    ax1 = fig.add_subplot(1,2,1) # Particle in a box
    ax1.set_yticklabels('')
    ax1.set_yticks([])
    ax1.set_xticklabels('')
    ax1.set_xticks([])
    ax1.set_aspect(1.0)
    
    ax1.scatter(x, y, c=colour)
    ax1.axis(xmin=x_low-3,xmax=x_high+3, ymin=y_low-3, ymax=y_high+3)
    
    
    
    ax2 = fig.add_subplot(1,2,2) # SIR counts scatter plot
    ax2.plot(time, sus, color="blue", label='Susceptible')
    ax2.plot(time, inf, color="red", label='Infected')
    ax2.plot(time, rec, color="green", label='Recovered')
    ax2.legend()
    ax2.set_xlabel("Time")
    ax2.set_ylabel("People")
    ax2.title.set_text("Change in SIR Numbers Across Time")

    
 # create the initial plot and append it to the frames list.        
frame=frame_plot(individuals, x_low= x_low,x_high= x_high, y_low=y_low, y_high=y_high, 
                 time=global_Frame, sus=global_S, inf=global_I, rec=global_R)
frames.append(frame)


for frame_num in range(num_frames):
    # num of SIR individuals per frame
    n_S=n
    n_I=0
    n_R=0
    for i in individuals: 
        # Check whether the infection period is up during the current frame
        i.getRecovered(frame_num)
        # Set new coordinates for the Individual        
        i.setNewPosition()
        # tallying SIR counts 
        if i.recovered:
            n_R+=1
            n_S-=1
        if i.infected:
            n_I+=1
            n_S-=1
            # Transmission
            for other in individuals:
                if other.identity == i.identity or other.infected or other.recovered:
                    pass
                else:
                    dist = i.getDist(other.coord_x, other.coord_y)
                    if dist < inf_radius:
                        if np.random.random() < inf_prob:
                            other.setInfected(frame_num)
    
    # Store this frames SIR values 
    global_S.append(n_S)
    global_I.append(n_I)
    global_R.append(n_R)
    global_Frame.append(frame_num+1) # Add one to account for the initial value created earlier. 
    
    # plot this frames values and store
    frame = frame_plot(individuals, 0, 100, 0, 100, 
                       time=global_Frame, sus=global_S, inf=global_I, rec=global_R)
    frames.append(frame)                       
    
    
# Creates a GIF from the frames list
gif.save(frames, "SIR_model.gif", duration=50)